# Car Advertisement Data Analysis
Welcome to the Vehicle Listing Data Analysis project! In this project, we delve into a dataset of used vehicles listed for sale to uncover interesting insights and trends. We explore various aspects of the data such as the relationship between vehicle price and model year, the distribution of vehicle conditions by model, and how long vehicles remain listed for sale.

### Analyzing data from car sales advertisement dataset.

In [48]:
# Loading all the libraries
import numpy as np
import pandas as pd
import plotly.express as px

In [49]:
# Load the data file
try:
    vehicles = pd.read_csv('../datasets/vehicles_us.csv')
except:
    vehicles = pd.read_csv('https://practicum-content.s3.us-west-1.amazonaws.com/datasets/vehicles_us.csv')

# Showing quick information about the data
vehicles.info()
display(vehicles.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
14182,1,2014.0,toyota 4runner,excellent,6.0,gas,NaN,automatic,SUV,blue,1.0,2018-06-23,61
36667,13600,2015.0,ford fusion,excellent,4.0,gas,43160.0,automatic,sedan,grey,NaN,2018-09-14,16
29159,3175,2005.0,chevrolet silverado,good,NaN,gas,205.0,automatic,pickup,silver,1.0,2018-05-29,45
835,6985,2015.0,nissan sentra,like new,4.0,gas,78044.0,automatic,sedan,NaN,NaN,2018-09-19,31
47071,8700,2013.0,chrysler 200,good,6.0,gas,74605.0,automatic,convertible,grey,NaN,2019-04-13,21


### Find and remove duplicate values

In [50]:
# Count the number of duplicated lines in the dataset.
print(vehicles.duplicated().sum())

0


Since there are no duplicates, we can start cleaning and fixing the data.

### Clean and Fix the Data

In [51]:
# Clean / Fix the data
# Convert 'date_posted' column into datetime type
vehicles['date_posted'] = pd.to_datetime(vehicles['date_posted'], format='%Y-%m-%d')

# Fill in missing values on 'model_year', 'odometer', 'is_4wd', and 'cylinders'
# with 0 and convert them into type int
vehicles['model_year'] = vehicles['model_year'].fillna(0)
if np.array_equal(vehicles['model_year'], vehicles['model_year'].astype('int')):
    vehicles['model_year'] = vehicles['model_year'].astype(int)
vehicles['odometer'] = vehicles['odometer'].fillna(0)
if np.array_equal(vehicles['odometer'], vehicles['odometer'].astype('int')):
    vehicles['odometer'] = vehicles['odometer'].astype(int)
vehicles['is_4wd'] = vehicles['is_4wd'].fillna(0)
if np.array_equal(vehicles['is_4wd'], vehicles['is_4wd'].astype('int')):
    vehicles['is_4wd'] = vehicles['is_4wd'].astype(int)
vehicles['cylinders'] = vehicles['cylinders'].fillna(0)
# if np.array_equal(vehicles['cylinders'], vehicles['cylinders'].astype('int')):
#     vehicles['cylinders'] = vehicles['cylinders'].astype(int)

# Calculate the median number of cylinders for each model and model_year
median_cylinders = vehicles.groupby(['model', 'model_year'])['cylinders'].transform('median')
print(median_cylinders.sample(10))

# Fill NaN values in the 'cylinders' column with the median values
vehicles['cylinders'].fillna(median_cylinders, inplace=True)

# Fill in missing paint_color values with 'unknown'
vehicles['paint_color'] = vehicles['paint_color'].fillna('unknown')

16794    8.0
14876    4.0
4577     6.0
10101    6.0
36685    6.0
33162    8.0
6958     4.0
1443     7.0
34239    6.0
51409    6.0
Name: cylinders, dtype: float64


In [52]:
# Split the model column so that the first word becomes the make and the rest
# is the actual model
vehicles[['make', 'model']] = vehicles['model'].str.split(' ', n=1, expand=True)
display(vehicles)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,make
0,9400,2011,x5,good,6.0,gas,145000,automatic,SUV,unknown,1,2018-06-23,19,bmw
1,25500,0,f-150,good,6.0,gas,88705,automatic,pickup,white,1,2018-10-19,50,ford
2,5500,2013,sonata,like new,4.0,gas,110000,automatic,sedan,red,0,2019-02-07,79,hyundai
3,1500,2003,f-150,fair,8.0,gas,0,automatic,pickup,unknown,0,2019-03-22,9,ford
4,14900,2017,200,excellent,4.0,gas,80903,automatic,sedan,black,0,2019-04-02,28,chrysler
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51520,9249,2013,maxima,like new,6.0,gas,88136,automatic,sedan,black,0,2018-10-03,37,nissan
51521,2700,2002,civic,salvage,4.0,gas,181500,automatic,sedan,white,0,2018-11-14,22,honda
51522,3950,2009,sonata,excellent,4.0,gas,128000,automatic,sedan,blue,0,2018-11-15,32,hyundai
51523,7455,2013,corolla,good,4.0,gas,139573,automatic,sedan,black,0,2018-07-02,71,toyota


In [53]:
# Create a list of unique car models
unique_makes = vehicles['make'].unique()

# Create a list of unique vehicle types
vehicle_types = vehicles['type'].unique()


In [54]:
# Create a DataFrame where model year is known (not filled with 0 from earlier)
vehicles_known_year = vehicles[vehicles['model_year'] != 0]


##### A scatterplot showing the median price of vehicles by type and model year

In [55]:
# First, we need to calculate the median price for each type and year
avg_price_type_year = vehicles_known_year.groupby(['type', 'model_year'])['price'].median().reset_index()

# Then, we create the scatter plot
fig = px.scatter(avg_price_type_year, x="model_year", y="price", color="type")
fig.update_layout(title_text='Median Price by Vehicle Type and Model Year', xaxis_title='Model Year', yaxis_title='Median Price')
fig.show()

The median price of vehicles has increased over time since about 2000. As a vehicle gets older than 2000, the price also generally increased. Additionally, it appears that SUVs, pickup, trucks, and off-roads have the highest median prices, while sedans, hatchbacks, and vans have the lowest median prices.

##### A scatterplot showing the median price of vehicles by brand and model year

In [56]:
# Now calculate the median price for each make and year
avg_price_make_year = vehicles_known_year.groupby(['make', 'model_year'])['price'].median().reset_index()

# Create the scatter plot
fig = px.scatter(avg_price_make_year, x="model_year", y="price", color="make")
fig.update_layout(title_text='Median Price by Vehicle Make and Model Year', xaxis_title='Model Year', yaxis_title='Median Price')
fig.show()

The median price of vehicles generally increases with the model year since around 2000. Luxury brands such as mercedes-benz and cadillac tend to have higher median prices than others, which indicates that the make of the vehicle has significant influcence on its price.

##### A histogram of days a vehicle is listed based on the car's make

In [57]:
# Create the histogram
fig = px.histogram(vehicles, x="days_listed", color="make")
fig.update_layout(title_text='Days Vehicle is Listed by Vehicle Make', xaxis_title='Days Listed', yaxis_title='Number of Vehicles Listed')
fig.show()

From this plot, it appears that the number of vehicles listed decreases as the number of days the vehicle is listed increases. This suggests that most vehicles are sold or removed from the listing within a short period of time. Most listings being there between 10 and 35 days.

##### A histogram of vehicle conditions based on the vehicle's model

In [58]:
# Create the histogram
fig = px.histogram(vehicles, x="condition", color="model", barmode='group')
fig.update_layout(title_text='Vehicle Condition by Model', xaxis_title='Condition', yaxis_title='Number of Vehicles')
fig.show()

Most people would consider the vehicle that they list as good or excellent condition. The highest number of listings seems to be for pickup trucks such as the F-150, Silverado 1500 and Ram 1500.

### Conclusion
The median price of vehicles are significantly affected by its model year and make. Luxury vehicles tend to have a higher median price vs non-luxury makes. Vehicles are mostly advertised for a short period of time. Trucks tend to be listed as excellent or good condition.
